In [1]:
import ReportMailer as rm

# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install xlsxwriter

# Reporting Period #
* PD - yesterday
* PW - previous week
* PM - previous month
* PQ - previous quarter
* PY - previous calendar year
* YTD year to date
* WTD current week to date
* TODAY today
* P90 - previous 90days

In [2]:
acronyms = [ "PD", "PW", "PM", "PQ", "PY", "YTD", "WTD", "TODAY", "P90"]
friendly = [ "Prior Day", "Prior Week", "Prior Month", "Prior Quarter", "Prior Year", "Year to Date", "Week to Date", "Today", "Previous 90 days"]

def toPeriodFriendly(INTERVAL):
    for n in range(0, len(acronyms)):
      if INTERVAL==acronyms[n]:
         return friendly[n]
    return "Unknown Period"

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution

def exit(): raise StopExecution

In [3]:
import os
import pandas as pd

path = "/home/gbadmin/jupy-notebooks/Reports/inqueue"
dir_list = os.listdir(path)

if len(dir_list)>0:
  filename = "inqueue/{}".format(dir_list[0])
  print(filename)

  rep = pd.read_json(filename, lines=True)

  for _ , row in rep.iterrows():    
    _INTERVAL = row['interval']
    _TYPE = row['type']
    if (_TYPE=='ALL'):
       _APPUSER  = row['appuser']
       print(_INTERVAL)
       print(_TYPE)
       # delete the file
       os.remove(filename)
    else:
      print("Non-Rollup type")
      exit()
       
else:
  print("NOTHING TO DO")
  exit()



NOTHING TO DO


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


#_INTERVAL = "PY"
_INTERVAL = 'YYYYMM202307'

In [2]:
from pandas import ExcelWriter
from pandas import ExcelFile
import xlsxwriter
import datetime

def createSpreadsheetAndMailIt(collections, reportname, recipients, subject, body):
  now = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')
  filename = reportname + "-" + now + ".xlsx"
  #writer = ExcelWriter(filename)
  writer = ExcelWriter(filename, engine='xlsxwriter')
  workbook = writer.book  
  print("Writing dataframe to Excel file {0}".format(filename))
  for collection in collections:
    print("Writing {} to spreadsheet".format(collection["name"]))
    collection["dataframe"].to_excel(writer, sheet_name=collection["name"], index=False)
    worksheet = writer.sheets[collection["name"]]
    if 'colwidths' in collection:
      colwidths = collection['colwidths']
      print("colwidths={}".format(colwidths))
      for n in range(0, len(colwidths)):
        col = str(chr(65 + n))
        worksheet.set_column('{}:{}'.format(col,col), colwidths[n])
    else:
      worksheet.set_column('A:A', 30)
  writer.save()
  for recipient in recipients: 
    rm.mailer(recipient, subject, body, filename)
  print("Excel/Email Done!")

In [3]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import GuestbookDbConnect as gdb

conn = gdb.guestbookDbConnect()

Welcome to Jupyter Notebook.  You are connected to the Opportunity House guestbook database!


In [12]:
import ReportTimerange as rtr

if _INTERVAL.startswith("YYYYMM"):
  yyyymm = _INTERVAL[6:12]
  print(yyyymm)
  yyyy = int(yyyymm[0:4])
  mm = int(yyyymm[4:6])
  print("{} {}".format(yyyy,mm))
  trange = rtr.mmyyyy(mm,yyyy)
else:
  trange = rtr.timerange(_INTERVAL)
print(trange)

collections = []
summaries = []

collection = {}
collection["name"] = "Outline"

df = pd.DataFrame({"Report Tab":[ \
              "Summary", \
              "Distinct Clients", \
              "Services", \
              "Housing", \
              "Incarcerations", \
              "Gender", \
              "Ethnicity", \
              "Veterans", \
              "New Clients"
                                 ], \
                   "Description":[ \
              "summary totals for the report period", \
              "a count of individuals served by sub-period within the period.", \
              "breakdown by service type and sub-period within the period.", \
              "breakdown by housing conditions and sub-period within the period.", \
              "clients reporting incarceration in previous 90 day period.", \
              "breakdown by gender and sub-period within the period.", \
              "breakdown by ethnicity and sub-period within the period.", \
              "breakdown by veteran status and sub-period within the period.", \
              "a list of clients with first visit occurring in the report period."
                                 ]})

collection["dataframe"] = df 
collection["colwidths"] = [30, 100]
collections.append(collection)

df.head(1000)

202307
2023 7
[datetime.datetime(2023, 7, 1, 0, 0), datetime.datetime(2023, 7, 31, 23, 59, 59, 999999), 'YYYY-MM']


,Report Tab,Description
0,Summary,summary totals for the report period
1,Distinct Clients,a count of individuals served by sub-period wi...
2,Services,breakdown by service type and sub-period withi...
3,Housing,breakdown by housing conditions and sub-period...
4,Incarcerations,clients reporting incarceration in previous 90...
5,Gender,breakdown by gender and sub-period within the ...
6,Ethnicity,breakdown by ethnicity and sub-period within t...
7,Veterans,breakdown by veteran status and sub-period wit...
8,New Clients,a list of clients with first visit occurring i...


## Total distinct clients in report period ##

In [8]:
summary = {}
summary["name"] = "Total distinct clients in report period"

query = "SELECT min(timestamp)AS start, max(timestamp) AS end, \
         COUNT(DISTINCT person_id) AS clients FROM guestbook_personsnapshot \
         WHERE timestamp BETWEEN '{}' AND '{}'".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

summary["count"] = data.iloc[0,2]
summaries.append(summary)
print(summaries)

data.head(100)

SELECT min(timestamp)AS start, max(timestamp) AS end,          COUNT(DISTINCT person_id) AS clients FROM guestbook_personsnapshot          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'
[{'name': 'Total distinct clients in report period', 'count': 732}]


,start,end,clients
0,2022-01-03 12:47:51.379347+00:00,2022-12-30 16:57:17.644126+00:00,732


# Distinct clients by sub-interval within the report period #
* Statistics focused on the breadth of individuals that are served during the time period.

In [9]:
collection = {}
collection["name"] = "Distinct Clients"

query = "SELECT to_char(timestamp,'{}') AS period,  \
         COUNT(DISTINCT person.idperson) AS clients \
         FROM guestbook_personsnapshot snapshot \
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period".format(trange[2], trange[0], trange[1])
print(query)

data = pd.read_sql(query, conn)
#data.head(1000)

persons = data.pivot_table('clients', index=['period']).fillna(0).astype(int).reset_index('period')

collection["dataframe"] = persons 
collection["colwidths"] = [30, 20]
collections.append(collection)
persons.head(1000)

SELECT to_char(timestamp,'YYYY-MM') AS period,           COUNT(DISTINCT person.idperson) AS clients          FROM guestbook_personsnapshot snapshot          JOIN guestbook_person person ON person.idperson=snapshot.person_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'          GROUP BY period


,period,clients
0,2022-01,157
1,2022-02,154
2,2022-03,173
3,2022-04,177
4,2022-05,195
5,2022-06,203
6,2022-07,203
7,2022-08,183
8,2022-09,174
9,2022-10,192


## Total Services delivered in report period ##

In [10]:
query = "SELECT \
         left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
         WHERE timestamp BETWEEN '{}' AND '{}'  AND service.points<=0 \
         GROUP BY servicename \
         ORDER BY total desc".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)


for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = data.iloc[n,0] + " services delivered."
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)
data.head(1000)

SELECT          left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot          JOIN guestbook_service service ON service.idservice=servicerequest.service_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'  AND service.points<=0          GROUP BY servicename          ORDER BY total desc
[{'name': 'Total distinct clients in report period', 'count': 732}, {'name': 'Meals services delivered.', 'count': 8911}, {'name': 'Shower services delivered.', 'count': 2860}, {'name': 'Clothing services delivered.', 'count': 1305}, {'name': 'Laundry services delivered.', 'count': 780}, {'name': 'Nurse services delivered.', 'count': 737}, {'name': 'Group Class services delivered.', 'count': 499}, {'name': 'Housing Assistance services delivered.', 'count': 185}, {'name': 'Pastor Consultat

,servicename,total
0,Meals,8911
1,Shower,2860
2,Clothing,1305
3,Laundry,780
4,Nurse,737
5,Group Class,499
6,Housing Assistance,185
7,Pastor Consultation,166
8,Computer,74
9,Toiletrie,9


# Services delivered by sub-interval within the report period #

In [11]:
collection = {}
collection["name"] = "Services"

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         service.name AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' \
#         GROUP BY period, servicename".format(trange[2], trange[0], trange[1])

query = "SELECT to_char(timestamp,'{}') AS period, \
         left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot \
         JOIN guestbook_service service ON service.idservice=servicerequest.service_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND service.points<=0 \
         GROUP BY period, servicename".format(trange[2], trange[0], trange[1])



print(query)

data = pd.read_sql(query, conn)

services = data.pivot_table('total', index=['period'], columns='servicename').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = services 
collection["colwidths"] = [30, 20, 20, 20, 20, 20, 20, 20]
collections.append(collection)

services.head(1000)

SELECT to_char(timestamp,'YYYY-MM') AS period,          left(service.name, strpos(service.name, '/') - 1) AS servicename, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personservicerequest servicerequest ON servicerequest.connection_id=snapshot.idsnapshot          JOIN guestbook_service service ON service.idservice=servicerequest.service_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999' AND service.points<=0          GROUP BY period, servicename


servicename,period,Clothing,Computer,Group Class,Housing Assistance,Laundry,Meals,Nurse,Pastor Consultation,Shower,Toiletrie
0,2022-01,85,13,16,27,48,592,57,12,185,0
1,2022-02,86,9,25,13,60,633,52,13,207,0
2,2022-03,84,4,17,6,83,722,68,16,253,1
3,2022-04,77,11,49,20,55,711,55,16,178,0
4,2022-05,95,2,81,21,61,798,87,20,215,0
5,2022-06,118,8,74,24,75,841,109,20,235,0
6,2022-07,119,6,31,13,55,711,86,23,221,3
7,2022-08,112,8,23,14,64,725,62,7,256,2
8,2022-09,129,2,62,8,63,731,55,8,272,2
9,2022-10,162,5,41,14,69,874,58,10,282,0


# Housing totals within the report period#

In [12]:
query = "SELECT  \
         left(housing.name, strpos(housing.name, '(') - 1)  AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
         GROUP BY hresponse".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = " Days spent in " + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(1000)

SELECT           left(housing.name, strpos(housing.name, '(') - 1)  AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999' AND prompt_id=8          GROUP BY hresponse
[{'name': 'Total distinct clients in report period', 'count': 732}, {'name': 'Meals services delivered.', 'count': 8911}, {'name': 'Shower services delivered.', 'count': 2860}, {'name': 'Clothing services delivered.', 'count': 1305}, {'name': 'Laundry services delivered.', 'count': 780}, {'name': 'Nurse services delivered.', 'count': 737}, {'name': 'Group Class services delivered.', 'count': 499}, {'name': 'Housing Assistance services delivered.', 'count': 185}, {'name': 'Pastor Consultation services delivered.', 'count': 166}, {'name': 'Computer ser

,hresponse,total
0,Unstable housing,1247
1,Jail or priso,11
2,Sheltered,2403
3,Stable housing,759
4,Unsheltered,5025


# Housing by City within the report period#

In [13]:
query = "SELECT  \
         city.name AS cresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_cityresponse city ON city.idcity=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=53 \
         GROUP BY cresponse ORDER BY total desc".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = " Nights spent in " + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(1000)

SELECT           city.name AS cresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_cityresponse city ON city.idcity=survey.object_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999' AND prompt_id=53          GROUP BY cresponse ORDER BY total desc
[{'name': 'Total distinct clients in report period', 'count': 732}, {'name': 'Meals services delivered.', 'count': 8911}, {'name': 'Shower services delivered.', 'count': 2860}, {'name': 'Clothing services delivered.', 'count': 1305}, {'name': 'Laundry services delivered.', 'count': 780}, {'name': 'Nurse services delivered.', 'count': 737}, {'name': 'Group Class services delivered.', 'count': 499}, {'name': 'Housing Assistance services delivered.', 'count': 185}, {'name': 'Pastor Consultation services delivered.', 'count': 166}, {'name': 'Computer services delivered.', 'count': 74},

,cresponse,total
0,Concord,6017
1,Kannapolis,731
2,Other,73


# Housing by sub-interval within the report period#
* Answers to the prompt 'Where did you spend last night?'

In [14]:
collection = {}
collection["name"] = "Housing"

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT to_char(timestamp,'{}') AS period, \
         left(housing.name, strpos(housing.name, '(') - 1) AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
#housing = housing.fillna(0)
collection["dataframe"] = housing  
collection["colwidths"] = [30, 30, 30, 30, 30, 30]
collections.append(collection)
housing.head(1000)

SELECT to_char(timestamp,'YYYY-MM') AS period,          left(housing.name, strpos(housing.name, '(') - 1) AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999' AND prompt_id=8          GROUP BY period, hresponse


hresponse,period,Jail or priso,Sheltered,Stable housing,Unsheltered,Unstable housing
0,2022-01,0,201,74,272,70
1,2022-02,0,172,83,340,65
2,2022-03,0,186,65,401,117
3,2022-04,1,228,89,353,106
4,2022-05,2,291,65,405,101
5,2022-06,3,276,68,459,109
6,2022-07,2,189,74,394,111
7,2022-08,0,155,61,449,105
8,2022-09,0,160,42,480,92
9,2022-10,1,132,46,571,163


# Housing by City and sub-interval within the report period#

In [15]:
collection = {}
collection["name"] = "Housing by City"

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT to_char(timestamp,'{}') AS period, \
         city.name AS cresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_cityresponse city ON city.idcity=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=53 \
         GROUP BY period, cresponse".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

city = data.pivot_table('total', index=['period'], columns='cresponse').fillna(0).astype(int).reset_index('period')
#housing = housing.fillna(0)
collection["dataframe"] = city  
collection["colwidths"] = [30, 30, 30, 30]
collections.append(collection)
city.head(1000)

SELECT to_char(timestamp,'YYYY-MM') AS period,          city.name AS cresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_cityresponse city ON city.idcity=survey.object_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999' AND prompt_id=53          GROUP BY period, cresponse


cresponse,period,Concord,Kannapolis,Other
0,2022-04,186,29,2
1,2022-05,753,103,2
2,2022-06,764,134,13
3,2022-07,693,75,2
4,2022-08,693,62,9
5,2022-09,676,93,6
6,2022-10,821,79,8
7,2022-11,774,101,11
8,2022-12,657,55,20


# Clients (by name) Reporting Unsheltered Housing #
* more than once in the report period

In [16]:
collection = {}
collection["name"] = "Unsheltered_by_Name"

unsheltered = 5
threshold = 1

#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT person.firstname, person.lastname, person.aliasname, \
         COUNT(*) AS unsheltered_nights FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 AND object_id={} \
         GROUP BY person.idperson ORDER BY COUNT(*) desc".format(trange[0], trange[1], unsheltered)

print(query)

data = pd.read_sql(query, conn)
data.head(100)

#housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = data 
collection["colwidths"] = [30, 30, 30, 30, 30, 30]
collections.append(collection)
data.head(1000)

SELECT person.firstname, person.lastname, person.aliasname,          COUNT(*) AS unsheltered_nights FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999' AND prompt_id=8 AND object_id=5          GROUP BY person.idperson ORDER BY COUNT(*) desc


,firstname,lastname,aliasname,unsheltered_nights
0,STEPHANIE,HELMS,STEPH,228
1,AARON,AYERS,AARONA,194
2,KAYSE,SIERRA,1234KAYSE_SI,182
3,CHARLEY,BLISSIT JR,KAKAROTT,174
4,DAVID,STIREWALT,HAWK,151
5,RANDY,HARRINGTON,RANDYH,136
6,TAMMY,COLLINS,0000TAMMY_CO,116
7,DAVID,QUERY,0000DAVID_QU,111
8,REGINALD,MOODY,REG,97
9,PAUL,MCDOWELL,0000PAUL_MCD,96


# Clients Reporting Incarceration in last 90 days #

In [17]:
collection = {}
collection["name"] = "Incarcerations"


#query = "SELECT to_char(timestamp,'{}') AS period, \
#         housing.name AS hresponse, COUNT(*) AS total FROM guestbook_personsnapshot snapshot\
#         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
#         JOIN guestbook_housingresponse housing ON housing.idhousing=survey.object_id \
#         WHERE timestamp BETWEEN '{}' AND '{}' AND prompt_id=8 \
#         GROUP BY period, hresponse".format(trange[2], trange[0], trange[1])

query = "SELECT person.idperson, person.firstname, person.lastname\
         FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot \
         WHERE survey.object_id=2 AND timestamp BETWEEN '{}' AND '{}' AND prompt_id=4 \
         GROUP BY person.idperson, person.firstname, person.lastname \
         ORDER BY person.idperson".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

#housing = data.pivot_table('total', index=['period'], columns='hresponse').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = data 
collection["colwidths"] = [30, 30, 30]
collections.append(collection)
data.head(1000)

SELECT person.idperson, person.firstname, person.lastname         FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_personsurvey survey ON survey.connection_id=snapshot.idsnapshot          WHERE survey.object_id=2 AND timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999' AND prompt_id=4          GROUP BY person.idperson, person.firstname, person.lastname          ORDER BY person.idperson


,idperson,firstname,lastname
0,24,CHRIS,MCKINNON
1,25,JAMIE,ERWIN
2,48,BRANDON,MILLER
3,56,BRANDON,AGUILAR
4,76,LARRY,LANE
5,99,RICK,HAMES
6,137,ALEX,LONG
7,147,TONY,SANDERS
8,169,LARRITT,SHEPPARD
9,173,TIMOTHY,KINDLEY


# Gender totals in the report period #

In [18]:
query = "SELECT  \
         gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY response".format(trange[0], trange[1])

print(query)
#summaries = []

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = "Gender -" + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(100)

SELECT           gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'          GROUP BY response
[{'name': 'Total distinct clients in report period', 'count': 732}, {'name': 'Meals services delivered.', 'count': 8911}, {'name': 'Shower services delivered.', 'count': 2860}, {'name': 'Clothing services delivered.', 'count': 1305}, {'name': 'Laundry services delivered.', 'count': 780}, {'name': 'Nurse services delivered.', 'count': 737}, {'name': 'Group Class services delivered.', 'count': 499}, {'name': 'Housing Assistance services delivered.', 'count': 185}, {'name': 'Pastor Consultation services delivered.', 'count': 166}, {'name': 'Computer services delivered.', 'count': 74}, {'name': 'Toiletr

,response,total
0,Female,89
1,Male,204
2,Unknown,439


# Gender by sub-interval within the report period #

In [19]:
collection = {}
collection["name"] = "Gender"

query = "SELECT to_char(timestamp,'{}') AS period, \
         gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period, response".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

gender = data.pivot_table('total', index=['period'], columns='response').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = gender
collection["colwidths"] = [30, 20, 20, 20]
#collection["colwidths"] = []
collections.append(collection)
gender.head(1000)

SELECT to_char(timestamp,'YYYY-MM') AS period,          gender.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_genderresponse gender ON gender.idgender=person.gender_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'          GROUP BY period, response


response,period,Female,Male,Unknown
0,2022-01,35,95,27
1,2022-02,37,90,27
2,2022-03,38,92,43
3,2022-04,36,93,48
4,2022-05,33,99,63
5,2022-06,36,90,77
6,2022-07,26,84,93
7,2022-08,32,79,72
8,2022-09,22,71,81
9,2022-10,20,76,96


# Ethnicity totals within the report period #

In [20]:
query = "SELECT  \
         ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY response".format(trange[0], trange[1])

print(query)
#summaries = []

data = pd.read_sql(query, conn)
for n in range(0, data.shape[0]):  
  summary = {}
  summary["name"]  = "Ethnicity -" + data.iloc[n,0] 
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)
data.head(1000)

SELECT           ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'          GROUP BY response
[{'name': 'Total distinct clients in report period', 'count': 732}, {'name': 'Meals services delivered.', 'count': 8911}, {'name': 'Shower services delivered.', 'count': 2860}, {'name': 'Clothing services delivered.', 'count': 1305}, {'name': 'Laundry services delivered.', 'count': 780}, {'name': 'Nurse services delivered.', 'count': 737}, {'name': 'Group Class services delivered.', 'count': 499}, {'name': 'Housing Assistance services delivered.', 'count': 185}, {'name': 'Pastor Consultation services delivered.', 'count': 166}, {'name': 'Computer services delivered.', 'count': 74},

,response,total
0,Asian,1
1,Black,173
2,Hispanic,6
3,Native American,5
4,Other,7
5,Unknown,389
6,White,151


# Ethnicity by sub-interval within the report period #

In [21]:
collection = {}
collection["name"] = "Ethnicity"

query = "SELECT to_char(timestamp,'{}') AS period, \
         ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period, response".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)

ethnicity = data.pivot_table('total', index=['period'], columns='response').fillna(0).astype(int).reset_index('period')
collection["dataframe"] = ethnicity 
collection["colwidths"] = [30, 20, 20, 20, 20, 20, 20, 20]
collections.append(collection)
ethnicity.head(1000)

SELECT to_char(timestamp,'YYYY-MM') AS period,          ethnicity.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_ethnicityresponse ethnicity ON ethnicity.idethnicity=person.ethnicity_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'          GROUP BY period, response


response,period,Asian,Black,Hispanic,Native American,Other,Unknown,White
0,2022-01,1,63,1,2,1,21,68
1,2022-02,0,61,1,3,3,18,68
2,2022-03,0,55,2,5,1,30,80
3,2022-04,0,60,2,2,1,38,74
4,2022-05,0,71,1,3,1,53,66
5,2022-06,0,70,0,2,2,64,65
6,2022-07,0,63,1,1,2,79,57
7,2022-08,0,56,1,3,4,60,59
8,2022-09,0,51,2,2,2,73,44
9,2022-10,0,56,2,2,2,85,45


# Veteran status totals within the report period#

In [22]:
query = "SELECT  \
         yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY response \
         ORDER BY response desc".format(trange[0], trange[1])

print(query)
#summaries = []

data = pd.read_sql(query, conn)

for n in range(0, data.shape[0]):  
  summary = {}
  if data.iloc[n,0]=="Yes":
    summary["name"] = "Veterans"
  elif data.iloc[n,0]=="No":
    summary["name"] = "Non-veterans"
  else:
    summary["name"] = "Veteran status Unknown"
  summary["count"] = data.iloc[n,1]
  summaries.append(summary)
print(summaries)

data.head(1000)

SELECT           yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'          GROUP BY response          ORDER BY response desc
[{'name': 'Total distinct clients in report period', 'count': 732}, {'name': 'Meals services delivered.', 'count': 8911}, {'name': 'Shower services delivered.', 'count': 2860}, {'name': 'Clothing services delivered.', 'count': 1305}, {'name': 'Laundry services delivered.', 'count': 780}, {'name': 'Nurse services delivered.', 'count': 737}, {'name': 'Group Class services delivered.', 'count': 499}, {'name': 'Housing Assistance services delivered.', 'count': 185}, {'name': 'Pastor Consultation services delivered.', 'count': 166}, {'name': 'Computer services delivered.', 'co

,response,total
0,Yes,15
1,Unknown,469
2,No,248


# Veteran Status by sub-interval within the report period#

In [23]:
collection = {}
collection["name"] = "Veterans"

query = "SELECT to_char(timestamp,'{}') AS period, \
         yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot\
         JOIN guestbook_person person ON person.idperson=snapshot.person_id \
         JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id \
         WHERE timestamp BETWEEN '{}' AND '{}' \
         GROUP BY period, response".format(trange[2], trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)
#data.head(1000)



veterans  = data.pivot_table('total', index=['period'], columns='response').fillna(0).astype(int).reset_index('period')
veterans.columns = ['period', 'non-veteran', 'unknown','veteran']
collection["dataframe"] = veterans 
collection["colwidths"] = [30, 30, 30, 30]
collections.append(collection)
veterans.head(1000)


SELECT to_char(timestamp,'YYYY-MM') AS period,          yesno.name AS response, COUNT(DISTINCT person.idperson) AS total FROM guestbook_personsnapshot snapshot         JOIN guestbook_person person ON person.idperson=snapshot.person_id          JOIN guestbook_yesnoresponse yesno ON yesno.idyesno=person.veteran_id          WHERE timestamp BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'          GROUP BY period, response


,period,non-veteran,unknown,veteran
0,2022-01,114,34,9
1,2022-02,113,33,8
2,2022-03,114,51,8
3,2022-04,114,56,7
4,2022-05,120,70,5
5,2022-06,116,82,5
6,2022-07,101,96,6
7,2022-08,101,77,5
8,2022-09,82,86,6
9,2022-10,84,100,8


# New Clients within the report period#
* Clients first appearing in the report interval

In [24]:
collection = {}
collection["name"] = "New Clients"

query = "SELECT firstname, lastname, aliasname, timelinestarttime::date AS startdate FROM guestbook_person \
         WHERE timelinestarttime BETWEEN '{}' AND '{}' \
         ORDER BY timelinestarttime asc".format(trange[0], trange[1])

print(query)

data = pd.read_sql(query, conn)
collection["dataframe"] = data
collection["colwidths"] = [30, 30, 30, 30]
collections.append(collection)

data.head(5000)

SELECT firstname, lastname, aliasname, timelinestarttime::date AS startdate FROM guestbook_person          WHERE timelinestarttime BETWEEN '2022-01-01 00:00:00' AND '2022-12-31 23:59:59.999999'          ORDER BY timelinestarttime asc


,firstname,lastname,aliasname,startdate
0,JOHNNY,PURSER,0000JOHNNY_P,2022-01-03
1,KRISTINNA,GLOVER,0000KRISTINN,2022-01-03
2,TINA,PARSONS,0000TINA_PAR,2022-01-03
3,MOHAMED,ALBASHIR,0000MOHAMED_,2022-01-04
4,CAGE,WALTER,0000CAGE_WAL,2022-01-04
5,JEREMY,SIMMONS,0000JEREMY_S,2022-01-06
6,SARAH,HEALEY,0000SARAH_HE,2022-01-06
7,EFREM,GAITHER,0000EFREM_GA,2022-01-10
8,JAMES,MILLER,0000JAMES_MI,2022-01-10
9,JAMIE,JENKINS,0000JAMIE_JE,2022-01-10


# Summary Counts #

In [25]:
collection = {}
summ = pd.DataFrame.from_dict(summaries)
collection["name"] = "Summary"
collection["dataframe"] = summ
collection["colwidths"] = [30, 60]
collections.insert(1, collection)
#collections.append(collection)
summ.head(1000)

,count,name
0,732,Total distinct clients in report period
1,8911,Meals services delivered.
2,2860,Shower services delivered.
3,1305,Clothing services delivered.
4,780,Laundry services delivered.
5,737,Nurse services delivered.
6,499,Group Class services delivered.
7,185,Housing Assistance services delivered.
8,166,Pastor Consultation services delivered.
9,74,Computer services delivered.


In [27]:
_REPORTNAME = _INTERVAL
_SUBJECT = "Opportunity House - Reports for {}".format(toPeriodFriendly(_INTERVAL))
_BODY    = "Spreadsheet (attached) with reports for period {} through {}.".format(trange[0], trange[1])
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com', 'jahood1@yahoo.com']
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com', 'pastor@opphouse.net']
#_EMAIL_RECIPIENT = ['cprice9739@carolina.rr.com']

query = "SELECT id, email FROM guestbook_appuser WHERE id={}".format(_APPUSER)

#print(query)
data = pd.read_sql(query, conn)
_EMAIL_RECIPIENT = {data.iloc[0,1]}
#_EMAIL_RECIPIENT = ['holobox@gmail.com']
print(">>{}<<".format(_EMAIL_RECIPIENT))

createSpreadsheetAndMailIt(collections, _REPORTNAME, _EMAIL_RECIPIENT, _SUBJECT, _BODY)

>>['holobox@gmail.com']<<
Writing dataframe to Excel file PY-2023-02-04_132914.xlsx
Writing Outline to spreadsheet
colwidths=[30, 100]
Writing Summary to spreadsheet
colwidths=[30, 60]
Writing Distinct Clients to spreadsheet
colwidths=[30, 20]
Writing Services to spreadsheet
colwidths=[30, 20, 20, 20, 20, 20, 20, 20]
Writing Housing to spreadsheet
colwidths=[30, 30, 30, 30, 30, 30]
Writing Housing by City to spreadsheet
colwidths=[30, 30, 30, 30]
Writing Unsheltered_by_Name to spreadsheet
colwidths=[30, 30, 30, 30, 30, 30]
Writing Incarcerations to spreadsheet
colwidths=[30, 30, 30]
Writing Gender to spreadsheet
colwidths=[30, 20, 20, 20]
Writing Ethnicity to spreadsheet
colwidths=[30, 20, 20, 20, 20, 20, 20, 20]
Writing Veterans to spreadsheet
colwidths=[30, 30, 30, 30]
Writing New Clients to spreadsheet
colwidths=[30, 30, 30, 30]
Emailer Done!
Excel/Email Done!
